In [124]:
import os
import json
import pprint
import re
import pkg_resources
import pandas as pd
import optparse
import sys
import hashlib

pp = pprint.PrettyPrinter(indent=4)

# GENERAL METHODS:
def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]

def convert_bytes(num):
    """
    this function will convert bytes to MB.... GB... etc
    """
    for x in ['bytes', 'KB', 'MB', 'GB', 'TB']:
        if num < 1024.0:
            return "%3.1f %s" % (num, x)
        num /= 1024.0

def get_file_size(file_path):
    """
    this function will return the file size
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return convert_bytes(file_info.st_size)

def set_motion_params(correct_motion=False,
                 ref_channel=0,
                 ref_file=0,
                 method=None,
                 algorithm=None
                ):

    mc_methods = ['Acquisition2P', 'NoRMCorre']
    mc_algos = dict((mc, []) for mc in mc_methods)
    mc_algos = {'Acquisition2P': ['@withinFile_withinFrame_lucasKanade', '@lucasKanade_plus_nonrigid'],
                'NoRMCorre': ['rigid', 'nonrigid']}

    if correct_motion is True:
        if method is None:
            while True:
                print "No MC method specified. Use default [Acquisition2P]?"
                mc_choice = raw_input('Enter <Y> to use default, or <o> to see options:')
                if mc_choice == 'Y':
                    print "Using default."
                    method = 'Acquisition2P'
                    break
                elif mc_choice == 'o':
                    for mcid, mcname in enumerate(mc_methods):
                        print mcid, mcname
                    mc_select = input('Enter IDX of motion-correction method to use:')
                    method = mc_methods[mc_select]
                    break
        if algorithm is None or (algorithm not in mc_algos[method]):
            print "No MC algorithm specified... Here are the options:"
            for algonum, algoname in enumerate(mc_algos[method]):
                print algonum, algoname
            algo_choice = input('Enter IDX of mc algorithm to use:')
            algorithm = mc_algos[method][algo_choice] 
    else:
        ref_channel = 0
        ref_file = 0
        method = None
        algorithm = None
        
    motion_params = dict()
    motion_params['correct_motion'] = correct_motion
    motion_params['method'] = method
    motion_params['algorithm'] = algorithm
    motion_params['ref_channel'] = ref_channel
    motion_params['ref_file'] = ref_file
    
    return motion_params

def set_preprocessing_params(correct_flyback=False,
                  nflyback_frames=0,
                  correct_bidir=False,
                  split_channels=False
                 ):
    
    preprocess_params = dict()
    if correct_flyback is False:
        nflyback_frames = 0
    
    preprocess_params['correct_flyback'] = correct_flyback
    preprocess_params['nflyback_frames'] = nflyback_frames
    preprocess_params['correct_bidir'] = correct_bidir
    preprocess_params['split_channels'] = split_channels
   
    return preprocess_params


def set_params(rootdir='', animalid='', session='', acquisition='', run='', tiffsource=None,
               correct_bidir=False, correct_flyback=False, nflyback_frames=None,
               split_channels=False, correct_motion=False, ref_file=1, ref_channel=1,
               mc_method=None, mc_algorithm=None):
    
    '''
    Create a dictionary of PARAMS used for processing TIFFs.
    PARAMS = {
        'source' : {
            'rootdir' (str)
                Root base of all data
            'animalid' : str
                Animal name (letters and numbers only, e.g., CE052)
            'session' (str)
                Session name (standard format: YYYYMMDD_<animalid>, e.g., 20171026_CE052)
            'acquisition': str
                Name of subdir in session directory for tiffs of a given FOV (e.g., FOV1_zoom3x')
            'run' (str)
                Name of experimental run (e.g., 'gratings_run1'). Child dir is 'raw' (acquired tiffs, read-only).
            'tiffsource' (str)
                Name of folder containing tiffs to be processed
                Full file-tree ex, <rootdir>/<animalid>/<session>/<acquisition>/<run>/raw
                }
        'motion' : {
            'correct_motion' (bool)
                True/False for whether to correct motion or not.
            'method' (str)
                Package to use for motion-correction (default: Acquisition2P)
                Options: Acquisition2P, NoRMCorre
            'algorithm' (str)
                Algorithm to use for motion-correction (default: '@withinFile_withinFrame_lucasKanade')
                Options:
                    Acquisition2P: '@withinFile_withinFrame_lucasKanade', '@lucasKanade_plus_nonrigid'
                    NoRMCorre: 'rigid', 'nonrigid'
            'ref_channel' (str)
                Reference channel to use for motion-correction (default: 1)
            'ref_file' (str)
                Reference file to use for motion-correction (default: 1)
            }
        'preprocessing' : {
            'correct_flyback' (bool)
                Whether to correct for bad flyback settings by removing extra frames from top of stack.
                Assumes stack acquisition starts from top (default: False)
            'nflyback_frames' (int)
                Number of extra frames to remove from the stack to correct flyback error (default: 1)
            'correct_bidir' (bool)
                Whether to correct scan phase if acquired with bidirectional scanning (default: False)
            'split_channels' (bool)
                Whether to split channels for extra large files (default: False)
                Will automatically check file size and set this to True if doing bidi- and motion-correction,
                since Matlab limit is 4GB.
                }
        'hashid' (str)
            Generated hash using the above key-value pairs, specific to PARAMS.
            Note: full PID dict includes extra house-keeping info that gets its own hash id.       
    }
    '''
    
    PARAMS = dict()
    PARAMS['source'] = dict()

    acquisition_dir = os.path.join(rootdir, animalid, session, acquisition)
    
    # Check tiffs to see if should split-channels for Matlab-based MC:
    if correct_motion is True or correct_bidir is True:
        rawdir_name = [r for r in os.listdir(os.path.join(acquisition_dir, run)) if 'raw' in r][0]
        rawtiff_dir = os.path.join(acquisition_dir, run, rawdir_name)
        tiffs = [t for t in os.listdir(rawtiff_dir) if t.endswith('tif')]
        file_sizes = [get_file_size(os.path.join(rawtiff_dir, t)) for t in tiffs]
        gb_files = [f for f in file_sizes if 'GB' in f]
        toobig = [g for g in gb_files if float(g.split('GB')[0]) > 4.0]
        if len(toobig) > 0:
            split_channels = True
        
    # ----------------------------------------------------------
    # Get source params:
    # ----------------------------------------------------------
    PARAMS['source']['rootdir'] = rootdir
    PARAMS['source']['animalid'] = animalid
    PARAMS['source']['session'] = session
    PARAMS['source']['acquisition'] = acquisition
    PARAMS['source']['run'] = run

    if not os.path.exists(os.path.join(acquisition_dir, run, 'processed')): 
        os.makedirs(os.path.join(acquisition_dir, run, 'processed'))

    processed_dirs = sorted([p for p in os.listdir(os.path.join(acquisition_dir, run, 'processed'))
                              if 'processed' in p], key=natural_keys)
 
    if tiffsource is None or len(tiffsource) == 0:
        while True:
            print "TIFF SOURCE was not specified."
            tiffsource_type = raw_input('Enter <P> if source is processed, <R> if raw: ')
            if tiffsource_type == 'R':
                tiffsource = 'raw'
                break
            elif tiffsource_type =='P':
                print "Selected PROCESSED source."
                pp.pprint(processed_dirs)
                if process_dict is None or (len(process_dict.keys()) == 0 and len(processed_dirs) == 0):
                    tiffsource = 'processed%03d' % int(len(processed_dirs) + 1)
                else:
                    candidate_sources = sorted(process_dict.keys(), key=natural_keys)
                    for pdix, pdir in enumerate(candidate_sources):
                        print pdix, pdir
                    tiffsource_idx = input('Select IDX of processed dir to use as source: ')
                    if tiffsource_idx < len(candidate_sources):
                        tiffsource = candidate_sources[int(tiffsource_idx)]
                confirm_selection = raw_input('Selected %s. Press <Y> to confirm.' % tiffsource)
                if confirm_selection == 'Y':
                    break
                    
    PARAMS['source']['tiffsource'] = tiffsource
    
    # ----------------------------------------------------------
    # Get preprocessing opts:
    # ----------------------------------------------------------
    PARAMS['preprocessing'] = set_preprocessing_params(correct_flyback=correct_flyback,
                             nflyback_frames=nflyback_frames,
                             correct_bidir=correct_bidir,
                             split_channels=split_channels)
    
    # ------------------------------------------
    # Check user-provided MC params:
    # ------------------------------------------
    print correct_motion
    PARAMS['motion'] = mcparams = set_motion_params(correct_motion=correct_motion,
                            ref_channel=ref_channel,
                            ref_file=ref_file,
                            method=mc_method,
                            algorithm=mc_algorithm)

    PARAMS['hashid'] = hashlib.sha1(json.dumps(PARAMS, sort_keys=True)).hexdigest()
    
    return PARAMS


def initialize_pid(PARAMS, acquisition_dir, run, auto=False):
    
    print "************************"
    print "Initialize PID."
    print "************************"
    process_id, is_new_pid, create_pid_file = get_process_id(PARAMS, acquisition_dir, run, auto=auto)

    pid = dict()
    version = pkg_resources.get_distribution('pipeline').version
    
    pid['version'] = version 
    pid['process_id'] = process_id
    pid['PARAMS'] = PARAMS
    pid['SRC'] = os.path.join(acquisition_dir, run, PARAMS['source']['tiffsource']) #source_dir
    pid['DST'] = os.path.join(acquisition_dir, run, 'processed', process_id)
    
    correct_flyback = pid['PARAMS']['preprocessing']['correct_flyback']
    correct_bidir = pid['PARAMS']['preprocessing']['correct_bidir']
    correct_motion = pid['PARAMS']['motion']['correct_motion']
    
    # Set source/dest dirs for Preprocessing tiffs:
    pid['PARAMS']['preprocessing']['sourcedir'] = pid['SRC']
    pid['PARAMS']['preprocessing']['destdir'] = None
    if correct_flyback is True:
        pid['PARAMS']['preprocessing']['destdir'] = os.path.join(pid['DST'], 'raw')
    if correct_bidir is True:
        pid['PARAMS']['preprocessing']['sourcedir'] = os.path.join(pid['DST'], 'raw')
        pid['PARAMS']['preprocessing']['destdir'] = os.path.join(pid['DST'], 'bidi')
       
    # Set source/dest dirs for Motion-Correction:
    pid['PARAMS']['motion']['sourcedir'] =  pid['PARAMS']['preprocessing']['destdir']
    pid['PARAMS']['motion']['destdir'] = None
    if correct_motion is True:
        if pid['PARAMS']['motion']['sourcedir'] is None:
            pid['PARAMS']['motion']['sourcedir'] = pid['SRC']
        pid['PARAMS']['motion']['destdir'] = os.path.join(pid['DST'], 'mcorrected')

    # TODO:  Generate hash for full PID dict
    tmp_hashid = hashlib.sha1(json.dumps(pid, sort_keys=True)).hexdigest()
    pid['tmp_hashid'] = tmp_hashid[0:6]
    
    return pid

def load_processdict(acquisition_dir, run):
    processdict = None
    processdict_filepath = os.path.join(acquisition_dir, run, 'processed', 'pid_info_%s.json' % run)
    
    # Load analysis "processdict" file:
    if os.path.exists(processdict_filepath):
        print "exists!"
        with open(processdict_filepath, 'r') as f:
            processdict = json.load(f)
            
    return processdict

def get_process_id(PARAMS, acquisition_dir, run, auto=False):
    
    process_id = None
    
    print "********************************"
    print "Checking previous PROCESS IDs..."
    print "********************************"
    
    # Load previously created PIDs:
    processdict = load_processdict(acquisition_dir, run)
    
    #Check current PARAMS against existing PId params by hashid:
    if processdict is None or len(processdict.keys()) == 0:
        processdict = dict()
        existing_pids = []
        is_new_pid = True
        print "No existing PIDs found."
        create_pid_file = True
    else:
        create_pid_file = False
        existing_pids = sorted([str(k) for k in processdict.keys()], key=natural_keys)
        matching_pids = sorted([pid for pid in existing_pids if processdict[pid]['PARAMS']['hashid'] == PARAMS['hashid']], key=natural_keys)
        is_new_pid = False
        if len(matching_pids) > 0:    
            print "Found match"
            while True:
                print "WARNING **************************************"
                print "Current param config matches existing PID:"
                for pidx, pid in enumerate(matching_pids):
                    print "******************************************"
                    print pidx, pid
                    pp.pprint(processdict[pid])
                if auto is True:
                    check_pidx = ''
                else:
                    check_pidx = raw_input('Enter IDX of pid to re-use, or hit <ENTER> to create new: ')
                if len(check_pidx) == 0:
                    is_new_pid = True
                    break
                else:
                    confirm_reuse = raw_input('Re-use PID %s? Press <Y> to confirm, any key to try again:' % existing_pids[int(check_pidx)])
                    if confirm_reuse == 'Y':
                        is_new_pid = False
                    break
        else:
            is_new_pid = True


    if is_new_pid is True:
        # Create new PID by incrementing num of process dirs found:
        process_id = 'processed%03d' % int(len(existing_pids)+1)
        print "Creating NEW pid: %s" % process_id
    else:
        # Re-using an existing PID:
        process_id = existing_pids[int(check_pidx)]
        print "Reusing existing pid: %s" % process_id
 
    return process_id, is_new_pid, create_pid_file


def get_default_pid(rootdir='', animalid='', session='', acquisition='', run='',
                    correct_flyback=None, nflyback_frames=0):
    
    acquisition_dir = os.path.join(rootdir, animalid, session, acquisition)
    
    DEFPARAMS = set_params(rootdir=rootdir, animalid=animalid, session=session,
                         acquisition=acquisition, run=run, tiffsource='raw',
                         correct_motion=False, correct_bidir=False,
                         correct_flyback=correct_flyback, nflyback_frames=nflyback_frames)
    
    # Generate new process_id based on input params:
    pid = initialize_pid(DEFPARAMS, acquisition_dir, run)
    
    # UPDATE RECORDS:
    update_records(pid, acquisition_dir, run)
    
    # STORE TMP FILE OF CURRENT PARAMS:
    tmp_pid_fn = 'tmp_pid_%s.json' % pid['hashid'][0:6]
    with open(os.path.join(acquisition_dir, run, 'processed', tmp_pid_fn), 'w') as f:
        json.dump(pid, f, indent=4, sort_keys=True)
        
    return pid

def update_records(pid, acquisition_dir, run):

    print "************************"
    print "Updating JSONS..."
    print "************************"

    processdict_filepath = os.path.join(acquisition_dir, run, 'processed', 'pid_info_%s.json' % run)
    if os.path.exists(processdict_filepath):
        with open(processdict_filepath, 'r') as f:
            processdict = json.load(f)
    else:
        processdict = dict()

    process_id = pid['process_id'] 
    processdict[process_id] = pid

    #% Update Process Info DICT:
    with open(processdict_filepath, 'w') as f:
        json.dump(processdict, f, sort_keys=True, indent=4)

    print "Process Info UPDATED."


### Set OPTPARSE params:

In [125]:
# rootdir = options.rootdir
# animalid = options.animalid
# session = options.session
# acquisition = options.acquisition
# run = options.run

# tiffsource = options.tiffsource

# # PREPROCESSING params:
# correct_bidir = options.bidi
# correct_flyback = options.flyback
# nflyback_frames = options.nflyback_frames

# # MOTION params:
# correct_motion = options.mc
# mc_method = options.mc_method
# mc_algorithm = options.algorithm
# ref_file = options.ref_file
# ref_channel = options.ref_channel

rootdir = '/nas/volume1/2photon/data' #options.rootdir
animalid = 'CE059' #options.animalid
session = '20171009_CE059' #options.session
acquisition = 'FOV1_zoom3x' #options.acquisition
run = 'gratings_run1' #options.run

tiffsource = 'raw' #options.tiffsource

# PREPROCESSING params:
correct_bidir = True #options.bidi
correct_flyback = True #options.flyback
nflyback_frames = 0 #options.nflyback_frames

# MOTION params:
correct_motion = True #options.mc
mc_method = None #options.mc_method
mc_algorithm = None #options.algorithm
ref_file = 0 #options.ref_file
ref_channel = 0 #options.ref_channel

In [126]:
    # Create config of PARAMS:
    PARAMS = set_params(rootdir=rootdir, animalid=animalid, session=session,
                            acquisition=acquisition, run=run, tiffsource=tiffsource,
                            correct_bidir=correct_bidir, correct_flyback=correct_flyback, nflyback_frames=nflyback_frames,
                            correct_motion=correct_motion, ref_file=ref_file, ref_channel=ref_channel, 
                            mc_method=mc_method, mc_algorithm=mc_algorithm)

True
No MC method specified. Use default [Acquisition2P]?
Enter <Y> to use default, or <o> to see options:Y
Using default.
No MC algorithm specified... Here are the options:
0 @withinFile_withinFrame_lucasKanade
1 @lucasKanade_plus_nonrigid
Enter IDX of mc algorithm to use:0


In [127]:
    acquisition_dir = os.path.join(rootdir, animalid, session, acquisition)
    # Generate new process_id based on input params:
    pid = initialize_pid(PARAMS, acquisition_dir, run)
    pp.pprint(pid)

************************
Initialize PID.
************************
********************************
Checking previous PROCESS IDs...
********************************
exists!
Creating NEW pid: processed002
{   'DST': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002',
    'PARAMS': {   'hashid': '06e4b48390af95a507582e5866837423f8f20b15',
                  'motion': {   'algorithm': '@withinFile_withinFrame_lucasKanade',
                                'correct_motion': True,
                                'destdir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/mcorrected',
                                'method': 'Acquisition2P',
                                'ref_channel': 0,
                                'ref_file': 0,
                                'sourcedir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/bidi'},
               

In [57]:
update_records(pid, acquisition_dir, run)

************************
Updating JSONS...
************************
Process Info UPDATED.


In [58]:
    # STORE TMP FILE OF CURRENT PARAMS:
    tmp_pid_fn = 'tmp_pid_%s.json' % pid['tmp_hashid'][0:6]
    tmp_pid_dir = os.path.join(acquisition_dir, run, 'processed', 'tmp_pids')
    if not os.path.exists(tmp_pid_dir):
        os.makedirs(tmp_pid_dir)
    with open(os.path.join(tmp_pid_dir, tmp_pid_fn), 'w') as f:
        json.dump(pid, f, indent=4, sort_keys=True)

In [128]:
pid

{'DST': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002',
 'PARAMS': {'hashid': '06e4b48390af95a507582e5866837423f8f20b15',
  'motion': {'algorithm': '@withinFile_withinFrame_lucasKanade',
   'correct_motion': True,
   'destdir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/mcorrected',
   'method': 'Acquisition2P',
   'ref_channel': 0,
   'ref_file': 0,
   'sourcedir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/bidi'},
  'preprocessing': {'correct_bidir': True,
   'correct_flyback': True,
   'destdir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/bidi',
   'nflyback_frames': 0,
   'sourcedir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/raw',
   'split_channels': False},
  'source': {'acquisition': 'FOV1_zoom3x',
   'animalid': '

In [129]:
pid['PARAMS']['source']['tiffsource']

'raw'

In [130]:
def append_hash_to_paths(PID, pid_hash, step=''):
    correct_flyback = PID['PARAMS']['preprocessing']['correct_flyback']
    correct_bidir = PID['PARAMS']['preprocessing']['correct_bidir']
    correct_motion = PID['PARAMS']['motion']['correct_motion']

    # If get_scanimage_data.py not run, <rundir>/raw/ does not have hash:
    if PID['PARAMS']['source']['tiffsource'] == 'raw':
        rawtiff_dir = PID['SRC']
        rawtiff_hashdir = [r for r in os.listdir(os.path.split(PID['SRC'])[0]) if 'raw_' in r]
        if len(rawtiff_hashdir)== 1:
            rawtiff_dir = os.path.join(os.path.split(PID['SRC'])[0], rawtiff_hashdir[0])
        elif os.path.exists(rawtiff_dir):
            # No hash created, yet, since renaming hasn't occured (default is 'raw')
            print "Checking tiffdir hash..."
            rawdir_hash = write_hash_readonly(rawtiff_hashdir, PID=None, step='simeta')
            if rawdir_hash not in rawtiff_dir:
                rawtiff_dir = rawtiff_dir + '_%s' % rawdir_hash
                print "Got hash for RAW dir:", rawtiff_dir
        PID['SRC'] = rawtiff_dir
        
    if pid_hash not in PID['DST']:
        PID['DST'] = PID['DST'] + '_%s' % pid_hash
    print 'PID %s: SRC is %s' % (pid_hash, PID['SRC'])
    print 'PID %s: DST is %s' % (pid_hash, PID['DST'])

    # Update source/dest, depending on current step and params:
    if step == 'flyback':
        PID['PARAMS']['preprocessing']['sourcedir'] = PID['SRC']
        if correct_flyback is True:
            PID['PARAMS']['preprocessing']['destdir'] = os.path.join(PID['DST'], 'raw')

    if step == 'bidir':
        if correct_flyback is True:
            # Bidir-correction is on flyback-corrected tiffs:
            PID['PARAMS']['preprocessing']['sourcedir'] = copy.copy(PID['PARAMS']['preprocessing']['destdir'])
            if '_' not in PID['PARAMS']['preprocessing']['sourcedir'] and\
                                os.path.isdir(PID['PARAMS']['preprocessing']['sourcedir']):
                PID = write_hash_readonly(PID['PARAMS']['preprocessing']['sourcedir'],
                                              PID=PID, step='preprocessing', label='bidir')
        else:
            # Bidir-correction is raw/SRC tiffs:
            PID['PARAMS']['preprocessing']['sourcedir'] = copy.copy(PID['SRC'])

        PID['PARAMS']['preprocessing']['destdir'] = os.path.join(PID['DST'], 'bidi')

    if step == 'motion':
        if correct_bidir is True:
            PID['PARAMS']['motion']['sourcedir'] = copy.copy(PID['PARAMS']['preprocessing']['sourcedir'])
        else:
            PID['PARAMS']['motion']['sourcedir'] = copy.copy(PID['SRC'])

        PID['PARAMS']['motion']['destdir'] = os.path.join(PID['DST'], 'mcorrected')

    return PID

In [131]:
def write_hash_readonly(write_dir, PID=None, step='', label=''):
    write_hash = None
    excluded_files = [str(f) for f in os.listdir(write_dir) if not f.endswith('tif')]
    print "Checking %s hash, excluded_files:" % label, excluded_files
    write_hash = dirhash(write_dir, 'sha1', excluded_files=excluded_files)[0:6]
    newwrite_dir = write_dir + '_%s' % write_hash
    
    # Rename dir if hash is not included:
    if write_hash not in write_dir:
        os.rename(write_dir, newwrite_dir)

    # Set READ-ONLY permissions:
    for f in os.listdir(newwrite_dir):
        os.chmod(os.path.join(newwrite_dir, f), S_IREAD|S_IRGRP|S_IROTH)  
        
    if PID is not None:
        if write_hash not in PID['PARAMS'][step]['destdir']:
            PID['PARAMS'][step]['destdir'] = newwrite_dir
        print "PID %s: Renamed output dir: %s" % (PID['tmp_hashid'], PID['PARAMS'][step]['destdir'])
    
    return write_hash, PID

In [132]:
pid

{'DST': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002',
 'PARAMS': {'hashid': '06e4b48390af95a507582e5866837423f8f20b15',
  'motion': {'algorithm': '@withinFile_withinFrame_lucasKanade',
   'correct_motion': True,
   'destdir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/mcorrected',
   'method': 'Acquisition2P',
   'ref_channel': 0,
   'ref_file': 0,
   'sourcedir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/bidi'},
  'preprocessing': {'correct_bidir': True,
   'correct_flyback': True,
   'destdir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/bidi',
   'nflyback_frames': 0,
   'sourcedir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/raw',
   'split_channels': False},
  'source': {'acquisition': 'FOV1_zoom3x',
   'animalid': '

In [133]:
pid_hash = 'd2342f'
pid_fb = append_hash_to_paths(pid, pid_hash, step='flyback')
pp.pprint(pid_fb)

PID d2342f: SRC is /nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/raw_b7b5d9
PID d2342f: DST is /nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002_d2342f
{   'DST': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002_d2342f',
    'PARAMS': {   'hashid': '06e4b48390af95a507582e5866837423f8f20b15',
                  'motion': {   'algorithm': '@withinFile_withinFrame_lucasKanade',
                                'correct_motion': True,
                                'destdir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/mcorrected',
                                'method': 'Acquisition2P',
                                'ref_channel': 0,
                                'ref_file': 0,
                                'sourcedir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processe

In [135]:
import copy
pid_bd = append_hash_to_paths(pid_fb, pid_hash, step='bidir')
pp.pprint(pid_bd)

PID d2342f: SRC is /nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/raw_b7b5d9
PID d2342f: DST is /nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002_d2342f
{   'DST': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002_d2342f',
    'PARAMS': {   'hashid': '06e4b48390af95a507582e5866837423f8f20b15',
                  'motion': {   'algorithm': '@withinFile_withinFrame_lucasKanade',
                                'correct_motion': True,
                                'destdir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processed002/mcorrected',
                                'method': 'Acquisition2P',
                                'ref_channel': 0,
                                'ref_file': 0,
                                'sourcedir': '/nas/volume1/2photon/data/CE059/20171009_CE059/FOV1_zoom3x/gratings_run1/processed/processe